In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from os import listdir



In [101]:
conflict_df = pd.read_csv('../data/conflict_data_irq.csv')

In [108]:
conflict_df.drop(0)

,id,year,active_year,start_year,end_year,type_of_violence,conflict_new_id,conflict_name,dyad_new_id,dyad_name,...,date_prec,date_start,date_end,deaths_a,deaths_b,deaths_civilians,deaths_unknown,low,best,high
1,114064.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-01-24,2014-01-25,0.0,0.0,1.0,7.0,8.0,8,8.0
2,62366.0,2013,1.0,2013,2013,3.0,506.0,IS - Civilians,973.0,IS - Civilians,...,1.0,2013-07-05,2013-07-05,0.0,0.0,0.0,7.0,7.0,7,7.0
3,114179.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-02-21,2014-02-22,6.0,0.0,0.0,0.0,6.0,6,6.0
4,114432.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-05-07,2014-05-08,0.0,0.0,0.0,7.0,7.0,7,7.0
5,143030.0,2014,1.0,2014,2014,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,2.0,2014-09-30,2014-10-01,7.0,7.0,0.0,0.0,14.0,14,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7469,277061.0,2018,1.0,2018,2018,1.0,354.0,Turkey: Kurdistan,781.0,Government of Turkey - PKK,...,1.0,2018-12-14,2018-12-14,0.0,3.0,0.0,0.0,3.0,3,3.0
7470,277062.0,2018,1.0,2018,2018,1.0,354.0,Turkey: Kurdistan,781.0,Government of Turkey - PKK,...,1.0,2018-12-14,2018-12-14,0.0,2.0,0.0,0.0,2.0,2,2.0
7471,274527.0,2018,1.0,2018,2018,1.0,354.0,Turkey: Kurdistan,781.0,Government of Turkey - PKK,...,2.0,2018-12-14,2018-12-15,0.0,7.0,0.0,0.0,0.0,7,7.0
7472,274534.0,2018,1.0,2018,2018,1.0,259.0,Iraq: Government,524.0,Government of Iraq - IS,...,1.0,2018-12-22,2018-12-22,0.0,4.0,0.0,0.0,4.0,4,4.0


In [52]:
returnee_filepaths = [f for f in listdir("../data/returnees/") if f.endswith('.xlsx')]


In [86]:
master_filepaths = [s for s in listdir("../data/master_list/") if s.endswith('.xlsx')]

In [90]:
mas_df = pd.concat((pd.read_excel("../data/master_list/"+s).assign(date=s[31:43]) 
                    for s in master_filepaths), ignore_index=True) 

In [91]:
mas_df

,Location ID,Place ID,Governorate,District,Location Name,Arabic Name,Latitude,Longitude,Families,Individuals,...,Post September 14 Period of displacement.1,Post March 16 Period of displacement,Post April15 Period of displacement.1,Post 17 October 16 Period of displacement,Post March 16 Period of displacement.1,July 17 Period of displacement,Post 17 October 16 Period of displacement.1,July 17 Period of displacement.1,Jan19,date
0,1101001.0,7861.0,Dahuk,Amedi,Amedi,عمادية,37.092389,43.487857,14.0,84.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jul 15, 2018"
1,1101002.0,8171.0,Dahuk,Amedi,Aradin,ارادن,37.108611,43.319828,9.0,54.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jul 15, 2018"
2,1101003.0,7870.0,Dahuk,Amedi,Ashawa,اشاوا,37.024985,43.294194,19.0,114.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jul 15, 2018"
3,1101004.0,8189.0,Dahuk,Amedi,Bamarne,بامرني,37.120577,43.268689,58.0,348.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jul 15, 2018"
4,1101005.0,7770.0,Dahuk,Amedi,Chamanke,جمانكي,36.937956,43.412792,102.0,612.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jul 15, 2018"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3736,3604015.0,22000.0,Qadissiya,Hamza,Hay Hussain,حي الحسين,31.716073,44.970706,77.0,462.0,...,29.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,"Feb 17, 2016"
3737,3604016.0,25284.0,Qadissiya,Hamza,Salim village,قرية سالم,31.736118,44.976184,40.0,240.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Feb 17, 2016"
3738,3604017.0,25281.0,Qadissiya,Hamza,Tabu,الطابو,31.659836,45.009918,27.0,162.0,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Feb 17, 2016"
3739,3604018.0,25282.0,Qadissiya,Hamza,Al-Taraf Al-Sharqi,الطرف الشرقي,31.719968,44.976969,30.0,180.0,...,10.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,"Feb 17, 2016"


In [92]:
mas_df['date'] = pd.to_datetime(mas_df['date'])

In [138]:
ret_df = pd.concat((pd.read_excel("../data/returnees/"+f).assign(date=f[35:47])
                    for f in filepaths), ignore_index=True)

In [139]:
ret_df

,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,Returnee Individuals,...,Pre June14 Period of displacement,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date
0,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,705.0,4230.0,...,0.0,0.0,705.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
1,1204011.0,13674.0,Erbil,Makhmur,Shahidan,شهيدان,35.780756,43.585956,510.0,3060.0,...,0.0,0.0,510.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
2,1204016.0,13501.0,Erbil,Makhmur,Kurdistan,كوردستان,35.767176,43.576238,280.0,1680.0,...,0.0,0.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
3,1204017.0,13675.0,Erbil,Makhmur,Sherwany,شيرواني,35.775643,43.584228,660.0,3960.0,...,0.0,0.0,660.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
4,1204018.0,27244.0,Erbil,Makhmur,Saray(markez bazar),سراي(مركز بازار),35.775451,43.577269,218.0,1308.0,...,0.0,0.0,218.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88360,2809030.0,27239.0,Salah al-Din,Tuz Khurmatu,Hay Al-Teen,حي التين,34.725080,44.580515,43.0,258.0,...,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
88361,2809031.0,27240.0,Salah al-Din,Tuz Khurmatu,Hay Sabaa Tamoz,حي سبعة تموز,34.724910,44.590226,20.0,120.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
88362,2809032.0,27241.0,Salah al-Din,Tuz Khurmatu,Hay Wahid Huzayran,حي واحد حزيران,34.719240,44.581716,30.0,180.0,...,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
88363,2809033.0,27242.0,Salah al-Din,Tuz Khurmatu,Qara Naz village,قرية قرناز,34.785884,44.570116,186.0,1116.0,...,170.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"


In [73]:
ret_df['date'] = pd.to_datetime(ret_df['date'])

In [112]:
sample = ret_df[ret_df['Location_name'] == 'Gwer']


In [131]:
ret_df

,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,Returnee Individuals,...,Pre June14 Period of displacement,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date
0,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,705.0,4230.0,...,0.0,0.0,705.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
1,1204011.0,13674.0,Erbil,Makhmur,Shahidan,شهيدان,35.780756,43.585956,510.0,3060.0,...,0.0,0.0,510.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
2,1204016.0,13501.0,Erbil,Makhmur,Kurdistan,كوردستان,35.767176,43.576238,280.0,1680.0,...,0.0,0.0,280.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
3,1204017.0,13675.0,Erbil,Makhmur,Sherwany,شيرواني,35.775643,43.584228,660.0,3960.0,...,0.0,0.0,660.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
4,1204018.0,27244.0,Erbil,Makhmur,Saray(markez bazar),سراي(مركز بازار),35.775451,43.577269,218.0,1308.0,...,0.0,0.0,218.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jan 05, 2017"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
469,2809030.0,27239.0,Salah al-Din,Tuz Khurmatu,Hay Al-Teen,حي التين,34.725080,44.580515,43.0,258.0,...,43.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
470,2809031.0,27240.0,Salah al-Din,Tuz Khurmatu,Hay Sabaa Tamoz,حي سبعة تموز,34.724910,44.590226,20.0,120.0,...,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
471,2809032.0,27241.0,Salah al-Din,Tuz Khurmatu,Hay Wahid Huzayran,حي واحد حزيران,34.719240,44.581716,30.0,180.0,...,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"
472,2809033.0,27242.0,Salah al-Din,Tuz Khurmatu,Qara Naz village,قرية قرناز,34.785884,44.570116,186.0,1116.0,...,170.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Aug 17, 2016"


In [116]:
sample.sort_values(by='date', ascending=True)

,Location ID,Place_ID,Governorate,District,Location_name,Arabic_name,Latitude,Longitude,Returnee Families,Returnee Individuals,...,Pre June14 Period of displacement,June July14 Period of displacement,August14 Period of displacement,Post September 14 Period of displacement,Post April15 Period of displacement,Post March 16 Period of displacement,Post 17 October 16 Period of displacement,July 17 Period of displacement,Jan19,date
0,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1141.0,6846.0,...,0.0,0.0,1141.0,0.0,0.0,0.0,0.0,0.0,0.0,"Apr 13, 2017"
1,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1478.0,8868.0,...,0.0,0.0,853.0,0.0,0.0,0.0,0.0,625.0,0.0,"Apr 15, 2018"
0,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1300.0,7800.0,...,0.0,0.0,1300.0,0.0,0.0,0.0,0.0,0.0,0.0,"Apr 27, 2017"
1,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,13356.0,...,0.0,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,"Apr 29, 2020"
1,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1480.0,8880.0,...,0.0,0.0,855.0,0.0,0.0,0.0,0.0,625.0,0.0,"Apr 30, 2018"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,2226.0,13356.0,...,0.0,0.0,1451.0,0.0,0.0,0.0,0.0,775.0,0.0,"Oct 31, 2020"
0,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1367.0,8202.0,...,0.0,0.0,1367.0,0.0,0.0,0.0,0.0,0.0,0.0,"Sep 15, 2017"
1,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1580.0,9480.0,...,0.0,0.0,955.0,0.0,0.0,0.0,0.0,625.0,0.0,"Sep 15, 2018"
0,1204004.0,13352.0,Erbil,Makhmur,Gwer,كوير,36.043816,43.499061,1367.0,8202.0,...,0.0,0.0,1367.0,0.0,0.0,0.0,0.0,0.0,0.0,"Sep 30, 2017"
